#### **Install required library packeges**


* ####  Using -- "*pip install*"




In [ ]:
!pip install "ibm-watsonx-ai==0.2.6"
!pip install "langchain" "langchain-core" "langchain-groq" "langchain-community"
!pip install "huggingface==0.0.1"
!pip install "huggingface-hub==0.21.4"
!pip install "sentence-transformers==2.5.1"
!pip install "chromadb==0.4.24"
!pip install "wget==3.2"
!pip install "groq"


## **Import Libraries**

In [5]:
def warn(*args, **kwargs):

    pass

import warnings
warnings.warn = warn
warnings.filterwarnings ('ignore')
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
import wget

from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os


# **Load the Document**

In [44]:
def read_file(filename):
  with open(filename, 'r') as document:
     content = document.read()
     print(content)

# Upload the  Document in colab and  open with  "file_path"
filename="House Lease Agreement.txt"

#File_path=input(': Enter the file path :_')
read_file(filename)

House Lease Agreement

This House Lease Agreement ("Agreement") is made on this 17th day of September, 2025, between:

Owner (Lessor): Mr. Ramesh Kumar  
Address: 12, Green Valley Apartments, Anna Nagar, Chennai, Tamil Nadu  
Phone: +91 98765 43210  

AND  

Tenant (Lessee): Mr. Arjun Mehta  
Address: 45, Sunrise Residency, Indiranagar, Bengaluru, Karnataka  
Phone: +91 91234 56789  

------------------------------------------------------------

1. Property Description
The Owner agrees to lease to the Tenant the residential property located at:
Flat No. 203, Lotus Residency, T. Nagar, Chennai – 600017

2. Lease Term
This lease shall begin on 1st October, 2025 and continue until 30th September, 2026, unless extended or terminated by mutual agreement.

3. Rent
The Tenant shall pay the Owner a monthly rent of ₹25,000, payable in advance on or before the 5th day of each month.

4. Deposit
A refundable security deposit of ₹75,000 shall be paid by the Tenant at the time of signing this Agree

# **Split the Document into chunks**

In [50]:
File_path='/content/House Lease Agreement.txt'
loader = TextLoader(File_path)
documents = loader.load()
with open(File_path, 'r', encoding='utf-8') as file:
            text = file.read()
text_splitter = CharacterTextSplitter(separator = "\n",chunk_size=1000, chunk_overlap=30)         #split the doc into chunks
texts = text_splitter.split_text(text)
#print(texts[0])
print(len(texts))

2


# **Embedding**

In [51]:
embed=HuggingFaceEmbeddings()
if texts:
  data=Chroma.from_texts(texts, embed)               # use chroma vector db to  store the vectors
print("------------- DONE -------------")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


------------- DONE -------------


# **Model Name : "Lamma - 3.2  "**

In [52]:
load_dotenv()
#api_key = os.environ.get("GROQ_API_KEY")   # using Groq api
api_key ="Enter the Groq API Key"

# Llm  part ---

My_llm= ChatGroq(groq_api_key=api_key, model_name="llama-3.3-70b-versatile")


# Ask  question repeatively

In [53]:
# to get the pipline flow  from file getting to responce makeing
import time

def pipe_line(file_path):

  loader = TextLoader(file_path)
  documents = loader.load()

  text_splitter = CharacterTextSplitter(separator='\n',chunk_size=500, chunk_overlap=20)
  texts = text_splitter.split_documents(documents)
  print(len(texts))

  embed=HuggingFaceEmbeddings()
  data=Chroma.from_documents(texts, embed)
  print("------------- DONE -------------")

  while True:
    query=input(f'--: Ask the Question : ')
    print('RESPONSE : - \n', get_response(query) )
    time.sleep(3)

In [54]:
def get_response(query):                                             # Intializa the  model to  get uses query and make responce
  qa = RetrievalQA.from_chain_type(
                               llm=My_llm,
                               chain_type="stuff",
                               retriever=data.as_retriever(),
                               )
  qa.return_source_documents = False
  response = qa.invoke(query)
  return response['result']

**Same Task**

In [55]:
while True:                                     # ask the query continously from the user
  query=input(f'\n\n---: Ask the Question : ')
  if (query=="no"):
    break
  else:

    print('\nRESPONSE :~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ \n', get_response(query) )
    print('\n============================================================================================\n')





---: Ask the Question : who is the owner

RESPONSE :~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
 The owner (Lessor) is Mr. Ramesh Kumar, and his address is 12, Green Valley Apartments, Anna Nagar, Chennai, Tamil Nadu, with a phone number of +91 98765 43210.




---: Ask the Question : no


# **Assignment-2**


In [56]:
import os
import re
import json
from transformers import pipeline
import json
import openpyxl
import pandas as pd



In [57]:
def load_and_preprocess(file_path):


  if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")
  with open(file_path, 'r', encoding='utf-8') as file:
      document = file.read()

  chunks = [document[i:i+1000] for i in range(0, len(document), 1000)]           #  split the doc into the chunks

  return chunks

In [58]:
def risk_detection(chunks):
    model_name = "google/flan-t5-base"                                       #load the model " google flan-t5"
    nlp = pipeline("text2text-generation", model=model_name)
    results=[]

#  pass the data to get the  risk and recommendations

    for chunk in chunks:
        prompt = ("Analyze the following  text   for potential risks, hidden obligations,"
               "or dependencies, and provide a recommendations format[where :  what :  ]:\n\n"+ chunk
               )
        result= nlp(prompt, max_length=200, do_sample=False)
        prompt = ("understand the following text for potential risks," "or dependencies, and provide risk detection format [Mistake:   ,issue will happen: ]:\n\n"+ chunk
              )
        result1= nlp(prompt, max_length=200, do_sample=False)
        results.append({"context": chunk, "recommendations": result[0]['generated_text'],"risk detection": result1[0]['generated_text']})


    return results

In [61]:
def main(file_path):

    chunks = load_and_preprocess(file_path)          #Loading the document

    print("Generating recommendations...")
    analysis = risk_detection(chunks)


    output_path = "risk_analysis.json"              # Save results to a JSON file

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(analysis, f, ensure_ascii=False, indent=4)
    print(f"Analysis complete. Results saved to {output_path}")





file_path = "/content/House Lease Agreement.txt"                                 #  path of document  to analysis
main(file_path)                                            #  call main funtion to  load and  detect the risk

Generating recommendations...


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_cla

Analysis complete. Results saved to risk_analysis.json


In [62]:

output_file = "output.xlsx"
with open("risk_analysis.json", "r", encoding="utf-8") as f:
    data = json.load(f)


df = pd.DataFrame(data)                                   # convert the data into frame  to store in excel
df.to_excel(output_file, index=False, engine='openpyxl')
print(f"Excel file created successfully at {output_file}")



for entry in data:
    print("Context:\n", entry["context"])
    print("Recommendation:\n-------------", entry["recommendations"])
    print("Risk Detection:\n-------------", entry["risk detection"])
    print("-" * 80)

Excel file created successfully at output.xlsx
Context:
 House Lease Agreement

This House Lease Agreement ("Agreement") is made on this 17th day of September, 2025, between:

Owner (Lessor): Mr. Ramesh Kumar  
Address: 12, Green Valley Apartments, Anna Nagar, Chennai, Tamil Nadu  
Phone: +91 98765 43210  

AND  

Tenant (Lessee): Mr. Arjun Mehta  
Address: 45, Sunrise Residency, Indiranagar, Bengaluru, Karnataka  
Phone: +91 91234 56789  

------------------------------------------------------------

1. Property Description
The Owner agrees to lease to the Tenant the residential property located at:
Flat No. 203, Lotus Residency, T. Nagar, Chennai – 600017

2. Lease Term
This lease shall begin on 1st October, 2025 and continue until 30th September, 2026, unless extended or terminated by mutual agreement.

3. Rent
The Tenant shall pay the Owner a monthly rent of ₹25,000, payable in advance on or before the 5th day of each month.

4. Deposit
A refundable security deposit of ₹75,000 shal

# **To send mail and save content in Google sheet**

In [63]:
path ="/infosys-449015-83ef8f804adb.json"
id="1rvEwtYh7mpqcBZgv7D14giGZ_mhBM-aiNZ_dxjJJQf0"

In [64]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email_simple(sender_email, sender_password, recipient_email, subject, body):
    try:
        # Set up the server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Start TLS encryption

        # Log in to your Gmail account
        server.login(sender_email, sender_password)

        # Create the email
        message = MIMEMultipart()
        message['From'] = sender_email
        message['To'] = recipient_email
        message['Subject'] = subject
        message.attach(MIMEText(body, 'plain'))

        # Send the email
        server.send_message(message)

        print("Email sent successfully!")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        server.quit()

if __name__ == "__main__":
    sender_email = "sender@gmail.com"
    sender_password = "Enter gmail App Key"
    recipient_email = "receiver@gmail.com"
    subject = "legal document  Risks and Recommendation "
    body = """Hello,
              This is for your doc  provided :  https://docs.google.com/spreadsheets/d/1rvEwtYh7mpqcBZgv7D14giGZ_mhBM-aiNZ_dxjJJQf0/edit?usp=sharing"""

    send_email_simple(sender_email, sender_password, recipient_email, subject, body)


Email sent successfully!


# **Mail alert about the Recent Law**

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import requests
from bs4 import BeautifulSoup
from langchain_groq import ChatGroq


#API_KEY = "Enter API Key"
#MODEL_NAME = "llama-3.2-11b-vision-preview"
llm = ChatGroq(groq_api_key=api_key, model_name="llama-3.3-70b-versatile")



#===========================================================================================================================================================
#===========================================================================================================================================================

def scrape_legal_news(url):
    print("Scrapping content ------------>>\n\n")
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    # Adjust selectors to target titles and links
    headlines = soup.select('h1, h2, h3, div')  # Update with appropriate selectors
    articles = []
    for headline in headlines:
        title = headline.get_text(strip=True)
        link_tag = headline.find('a')
        link = link_tag['href'] if link_tag and 'href' in link_tag.attrs else None
        if title:  # Ensure title is not empty
            articles.append({"title": title, "link": link or "No link available"})
    return articles



#===========================================================================================================================================================
#===========================================================================================================================================================


def summarize_news_in_chunks(news_list, max_chunk_size=7000):

    if not news_list:
        return "No news articles to summarize."

    chunks = []
    current_chunk = ""
    chunk_size = 0

    # Split the news into chunks based on max token size
    for i, news in enumerate(news_list, 1):
        title = news.get('title', 'No title available')
        link = news.get('link', 'No link available')
        news_item = f"{i}. {title} ({link})\n"
        token_count = len(news_item)  # Approximate token count using character length

        # If adding this news exceeds the chunk size, start a new chunk
        if chunk_size + token_count > max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = news_item
            chunk_size = token_count
        else:
            current_chunk += news_item
            chunk_size += token_count

    # Append the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    # Summarize each chunk using the Llama model
    print("Model call...\n\n")

    summaries = []
    for chunk in chunks:
        input_text = f"Summarize the following legal updates into key sentences: more simpler in 2 lines \n{chunk}"
        response = llm.invoke(input_text)  # Call Llama model
        summaries.append(response.content)

    # Combine all summaries into one
    return "\n\n".join(summaries)



#===========================================================================================================================================================
#===========================================================================================================================================================


# Step 3: Send the email
def send_email(subject, body, recipient_email, sender_email, sender_password):
    print("Sending Mail...\n\n")
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'html'))

    # Connect to the SMTP server
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient_email, msg.as_string())





#===========================================================================================================================================================
#===========================================================================================================================================================





def main():
    # legal_news_url = input()
    legal_news_url = "https://economictimes.indiatimes.com/wealth/legal/will/rent-agreements-must-have-these-clauses-to-protect-landlords-rights/articleshow/104571151.cms?from=mdr"
    recipient_email = "receiver@gmail.com"
    sender_email = "sender@gmail.com"
    sender_password = "Enter gmail App Key"


    # Scrape and summarize news
    scraped_news = scrape_legal_news(legal_news_url)
    if not scraped_news:
        print("No news found.")
        return


    summarized_news = summarize_news_in_chunks(scraped_news)

    # Create email body with alignment
    email_body = "<html><body>"
    email_body += "<h2>Recent Legal Updates</h2>"
    email_body += "<ul>"
    for point in summarized_news.split('\n'):
        email_body += f"<li>{point.strip()}</li>"
    email_body += "</ul>"
    email_body += "</body></html>"



    # Send email
    send_email(
        subject="Important Legal Updates",
        body=email_body,
        recipient_email=recipient_email,
        sender_email=sender_email,
        sender_password=sender_password
    )

    print("Email sent successfully!")




if __name__ == "__main__":
    main()
